In [4]:
import sqlite3
import pandas as pd

In [5]:
conn = sqlite3.connect("../data/twitter.db")
cursor = conn.cursor()

In [7]:
data = cursor.execute("SELECT * from tweets_raw;")
result = cursor.fetchall()
#author_id = result[0][0]

In [8]:
result

[(1542138596387438593,
  19067150,
  "#Python Pop Quiz 🐍❓\n\nHere we have a dataclass where I want to have a default list. But it doesn't work! Can you fix the code so it works? https://t.co/wmWKeAlVjh"),
 (1542071124737024002,
  19067150,
  '@mousevspython Thanks for checking out my mini-thread on `collections.ChainMap`!\n\nPython is an amazing language. Follow me to learn more! 🐍🔥'),
 (1542071120588939264,
  19067150,
  'You can learn more about how the `ChainMap` in my tutorial @mousevspython\n\nhttps://t.co/AaydrrGPXc'),
 (1542071115928977409,
  19067150,
  "Here's another example where you create a ChainMap that looks at the following mappings:\n\n🐍  Command line arguments\n🐍  Environment variables\n🐍  A default mapping\n\nThis allows the user to override settings on the command line or fall back to env vars or the defaults https://t.co/cW4Gfuoywq"),
 (1542071111474634754,
  19067150,
  "Let's look at an example of using `ChainMap` in #Python https://t.co/bEBGeYQOu0"),
 (154207110

In [10]:
from ..src.mockingjay.process import Process

ImportError: attempted relative import with no known parent package

In [15]:
tweets = cursor.execute("SELECT tweetID, tweet FROM tweets WHERE authorID = (?)", (author_id,))
data = cursor.fetchall()

In [78]:
min_words = 3
df = pd.DataFrame(data, columns=["id", "text"])



clean_data(df)

,id,text
0,1535548780535173120,best fake cryptic cube
1,1535548397158969344,close your door
2,1535547131645157376,what is your name
3,1535546957107560448,the game only a human could randomly make up s...
4,1535546828988379137,you gotta learn to take a joke
...,...,...
488,1534978313252327424,value is a construct
489,1534978080271323137,a moment of silence☯️😌💫
490,1534977934347227136,and save the world
491,1534977905796608000,lol have a seance
